### CISC 452 Assignment 2

### Imports

In [0]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

from random import random
from math import exp
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, precision_score, recall_score

### Get the Data

In [3]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

def normalize(x):
    return x / 255.

def flatten(x):
    return x.reshape(-1, np.prod(x.shape[1:]))

def preprocess(x):
    return normalize(flatten(x))


train_x = preprocess(train_x)
test_x = preprocess(test_x)

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
output_file = open("output_file.txt", 'w')
output_file.write("""********************
Mitch Mathieu
10157108
CISC 452
Assignment 2
********************\n\n""")

88

## The Network

In [0]:
class Network:
  def __init__(self, epochs, l_rate, momentum, num_inputs, num_hidden, num_outputs):
    self.epochs = epochs
    self.learning_rate = l_rate
    self.momentum = momentum
    self.num_outputs = num_outputs
    output_file.write(f"Number of inputs: {num_inputs}\n")
    output_file.write(f"Number of hidden nodes: {num_hidden}\n")
    output_file.write(f"Number of output nodes: {num_outputs}\n")
    # the network is a list of lists of dictionaries
    # a list of dictionaries represents a layer
    # a dictionary represents a neuron
    self.network = []
    hidden_layer = [{'weights': [random() for i in range(num_inputs + 1)]} for i in range(num_hidden)]
    self.network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(num_hidden + 1)]} for i in range(num_outputs)]
    self.network.append(output_layer)
  
  
  # determine the activation of the neuron
  def activate(self, weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
      activation += weights[i] * inputs[i]
    return activation
  
  
  # sigmoid function
  def sigmoid(self, activation):
    return 1.0 / (1 + exp(-activation))
  
  
  # derivative of sigmoid function
  def sigmoid_derivative(self, output):
    return output * (1 - output)
  
  
  # use momentum and the deltas to update the weights
  def update_weights(self, row): 
    # row is a set of inputs
    for i in range(len(self.network)):
      inputs = row[:-1]
      if i != 0:
        inputs = [neuron['output'] for neuron in self.network[i - 1]]
      for neuron in self.network[i]:
        for j in range(len(inputs)):
          neuron['weights'][j] += self.learning_rate * neuron['delta'] * inputs[j]
          if 'previous delta' in neuron:
            neuron['weights'][j] += (self.momentum * neuron['previous delta'])
        neuron['weights'][-1] += self.learning_rate * neuron['delta']
  
  
  # work back from the output layer and assign errors and deltas
  def backward_propagation(self, expected):
    for i in reversed(range(len(self.network))):
      layer = self.network[i]
      errors = []
      # check if hidden layer
      if i != len(self.network) - 1:
        for j in range(len(layer)):
          error = 0.0
          for neuron in self.network[i + 1]:
            error += (neuron['weights'][j] * neuron['delta'])
          errors.append(error)
      # output layer
      else:
        for j in range(len(layer)):
          neuron = layer[j]
          errors.append(expected[j] - neuron['output'])
          
      for j in range(len(layer)):
        # add deltas to each neuron in current layer
        neuron = layer[j]
        if 'delta' in neuron:
            neuron['previous delta'] = neuron['delta']
        neuron['delta'] = errors[j] * self.sigmoid_derivative(neuron['output'])
  
  
  # use the current weights and input to determine output
  def forward_propagation(self, row):
    inputs = row
    for layer in self.network:
      new_inputs = []
      for neuron in layer:
        activation = self.activate(neuron['weights'], inputs)
        neuron['output'] = self.sigmoid(activation)
        new_inputs.append(neuron['output'])
      inputs = new_inputs
    return inputs
  
  
  def train(self, train_x, train_y):
    for epoch in range(self.epochs):
      count = 0
      
      print("**********************")
      print(f"Starting epoch {epoch+1}")
      print("**********************")
      
      sum_error = 0
      for row, label in zip(train_x, train_y):
        outputs = self.forward_propagation(row)
        expected = label
        sum_error += sum([(expected[i] - outputs[i])**2 for i in range(expected.size)])
        self.backward_propagation(expected)
        self.update_weights(row)
        
        count += 1
        if count % 10000 == 0:
          print(count)
          
      print(f">epoch={epoch+1} error={sum_error:.3f}")
      
  
  def predict(self, row):
    outputs = self.forward_propagation(row)
    index = outputs.index(max(outputs))
    result = [0 for i in range(self.num_outputs)]
    result[index] = 1
    return result
  
  
  def vec_to_cat(self, vec):
    for i in range(len(vec)):
      if vec[i] == 1:
        return i
      
      
  def test(self, test_x, test_y):
    correct = 0
    predictions = []
    for row, label in zip(test_x, test_y):
      prediction = self.predict(row)
      predictions.append(self.vec_to_cat(prediction))
      if np.array_equal(prediction, label):
        correct += 1
    
    print(f"\nAccuracy: {correct/test_x.size:.4f}%")
    return predictions

### Instantiate and Train the Network

In [7]:
# params: epochs, l_rate, momentum, num_inputs, num_hidden, num_outputs
nn = Network(3, 0.5, 0.5, 784, 5, 10)

train_y_enc = np.zeros((train_y.size, train_y.max()+1))
train_y_enc[np.arange(train_y.size), train_y] = 1

nn.train(train_x, train_y_enc)

**********************
Starting epoch 1
**********************
10000
20000
30000
40000
50000
60000
>epoch=1 error=55193.294
**********************
Starting epoch 2
**********************
10000
20000
30000
40000
50000
60000
>epoch=2 error=55123.883
**********************
Starting epoch 3
**********************
10000
20000
30000
40000
50000
60000
>epoch=3 error=55123.883


In [8]:
test_y_enc = np.zeros((test_y.size, test_y.max()+1))
test_y_enc[np.arange(test_y.size), test_y] = 1

predictions = nn.test(test_x, test_y_enc)


Accuracy: 0.0001%


In [9]:
cm = confusion_matrix(test_y, 
                      predictions, 
                      labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

precision = precision_score(test_y, 
                            predictions, 
                            labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                            average='micro')

recall = recall_score(test_y, 
                      predictions, 
                      labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                      average='micro')
# print(cm)
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")

output_file.write("\n\nResults:\n")
output_file.write("Confusion Matrix:\n")
output_file.write(f"{cm}\n\n")
output_file.write(f"Precision: {precision:.4f}\n")
output_file.write(f"Recall: {recall:.4f}\n")

15

In [0]:
output_file.close()

In [13]:
! cat output_file.txt

********************
Mitch Mathieu
10157108
CISC 452
Assignment 2
********************

Number of inputs: 784
Number of hidden nodes: 5
Number of output nodes: 10


Results:
Confusion Matrix:
[[   0    0    0    0    0    0  980    0    0    0]
 [   0    0    0    0    0    0 1135    0    0    0]
 [   0    0    0    0    0    0 1032    0    0    0]
 [   0    0    0    0    0    0 1010    0    0    0]
 [   0    0    0    0    0    0  982    0    0    0]
 [   0    0    0    0    0    0  892    0    0    0]
 [   0    0    0    0    0    0  958    0    0    0]
 [   0    0    0    0    0    0 1028    0    0    0]
 [   0    0    0    0    0    0  974    0    0    0]
 [   0    0    0    0    0    0 1009    0    0    0]]

Precision: 0.0958
Recall: 0.0958


In [0]:
from google.colab import files
files.download('output_file.txt')